<a href="https://colab.research.google.com/github/adititadkod15-tech/HinglishLID1/blob/main/Qwen_Qwen3_14B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import gc
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
# Clear the "Ghost" memory from the failed 30B load
gc.collect()
torch.cuda.empty_cache()

In [3]:
!pip install -U bitsandbytes

In [4]:
!pip install flash-attn --no-build-isolation

Please restart your Colab runtime after this installation for the changes to take full effect.

In [5]:


model_id = "Qwen/Qwen3-14B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="flash_attention_2"
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Please restart your Colab runtime after this installation for the changes to take full effect.

In [27]:
# Use Qwen3-8B for speed and zero memory crashes
# Change line 3 in your top block to:
model_id = "Qwen/Qwen3-14B"

system_prompt = """Act as a strict Linguistic Filter. Your ONLY job is to transliterate Roman-Hindi words to Devanagari.

### ABSOLUTE CONSTRAINTS:
1. SILENT PURGE: If a word is English (e.g., 'war', 'directly', 'biological', 'chemical', 'weapons', 'missile', 'bomb', 'roundtable', 'discussion'), do NOT output it. Do NOT translate it. Do NOT transliterate it.
2. NO PROPER NOUNS: Delete names of places or groups (e.g., 'India', 'Cambodian', 'Naxalite', 'Maoist').
3. NO NOTES: Do not provide "(delete)" tags or summaries. Output only the word pairs.
4. NO TRANSLATION: Never turn 'weapons' into 'हथियार'. If you see 'weapons', it must be DELETED from the output list.

### FORMAT:
[original_roman] -> [devanagari]

### EXAMPLES:
Input: 'wars ke beech chemical weapons'
Output:
ke -> के
beech -> बीच

Input: 'missile ya bomb se lakhon log'
Output:
ya -> या
se -> से
lakhon -> लाखों
log -> लोग"""
def get_extraction(sentence):
    # Ensure messages use the Instruct format for Qwen3-14B
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"INPUT: {sentence}"}
    ]

    # Disable thinking mode to prioritize structural accuracy over reasoning
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Use greedy decoding (do_sample=False) to stop 'translation' hallucinations
    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=False,
        repetition_penalty=1.1,
        # Ensure BFloat16 precision for A100 hardware
        pad_token_id=tokenizer.eos_token_id
    )

    # Extract only the model's response
    response = tokenizer.decode(outputs[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
    return response.strip()


In [28]:
sample_sentences =[
    'Pahle toh hum baat karte hain Cambodian Civil War ki (999 words). Wahan par jo hua, woh sach mein heartache aur shok se bhara tha. Lakhon logon ki zindagiyan barbad ho gayihi. Kitni hi auraten vidhvaa ho gayihi, aur kitne bachche anath! Sab ke chehron par sirf nirasha aur melancholic hi nazar aata hoga us samay.',
    'Mujhe yaad hai jab maine ek documentary dekha tha is war ke baare mein. Woh sab sunkar mera rooh kaanp gaya tha. Bachchon ko apne maa-baap ko khote dekhna, auraton ki vidhvavastha, ye sab dekhkar ekdum se mera man udaas aur dismay se bhar gaya tha. ',
    'Jaisa ki aap sab jaante hain, wars ke beech chemical aur biological weapons ka istemaal bhi hota hai. Inse toh aur bhi nuksan hota hai zindagiyon ka. Ek missile ya bomb se lakhon log dukhi ho jaate hain - apne ghar gawaa dete hain, apne parivaar ko khaa dete hain. Ye sab sochna hi itna hidaayatheen karta hai',
    'Aaj ki is roundtable discussion ka mukhy vishay bahut hi bhavuk hai (999 words) - wars aur conflicts ke emotional asar par charcha karni hai. Ye ek aisi cheez hai jo har insaan ko chhoo kar jaati hai, chahe woh usmein directly shaamil ho ya na ho.',
    'Fijian Coups ke baare mein padha tha maine kuch dino pehle. Wahan par bhi bahut kuchh hua tha. Aakhir kyun hoti hain ye sab cheezein? Kyun koi use bombs aur weapons karta hai doosre ki zindagiyon ko barbad karne ke liye? Ye sab jaanne ke baad toh main ekdum hi nimhans ho gaya tha.',
    'Aur jaisa ki hum jaante hain, India bhi aise hi kayi wars aur insurgencies se guzra hai. Jaise ki Naxalite-Maoist insurgency, aur Eritrean War of Independence. In dono mein kitne hi jawaan shaheed hue, aur kitne gareeb logon ko apni zindagiyaan gawani padin. Sab ke ghar barbad ho gaye, sari umeeden toot gayihi.',
    'War politics ke peeche badi wajah ye fear of war hota hai har country ka (999 words). Har koi chahta hai ki unhe jeetna hai, unhe power mein rehna hai. Issi liye ye sab wars shuru hote rehte hain. Par in sab ke peeche, sabse bada nuksan toh aam insaanon ka hi hota hai. ',

]

for sample_sentence in sample_sentences:
  print(sample_sentence)
  print('\n')
  result = get_extraction(sample_sentence)
  print(result)

Pahle toh hum baat karte hain Cambodian Civil War ki (999 words). Wahan par jo hua, woh sach mein heartache aur shok se bhara tha. Lakhon logon ki zindagiyan barbad ho gayihi. Kitni hi auraten vidhvaa ho gayihi, aur kitne bachche anath! Sab ke chehron par sirf nirasha aur melancholic hi nazar aata hoga us samay.


Pahle -> पहले  
toh -> तो  
hum -> हम  
baat -> बात  
karte -> करते  
hain -> हैं  
ki -> कि  
wahan -> वहां  
par -> पर  
jo -> जो  
hua -> हुआ  
woh -> वो  
sach -> सच  
mein -> में  
heartache -> हार्टएच  
aur -> और  
shok -> शोक  
se -> से  
bhara -> भरा  
tha -> था  
lakhon -> लाखों  
logon -> लोगों  
ki -> कि  
zindagiyan -> ज़िन्दगियाँ  
barbad -> बरबद  
ho -> हो  
gayihi -> गईहि  
kitni -> कितनी  
hi -> ही  
auraten -> अवरतन  
vidhvaa -> विधवा  
ho -> हो  
gayihi -> गईहि  
aur -> और  
kitne -> कितने  
bachche -> बच्चे  
anath -> अनाथ  
Sab -> सब  
ke -> के  
chehron -> छोहर  
par -> पर  
sirf -> सिर्फ  
nirasha -> निराशा  
aur -> और  
melancholic -> मेलांचोलिक  
hi ->